In [22]:
import pickle
import pandas as pd

In [23]:
from pathlib import Path
dir = Path('.')
check = dir / 'info.pkl'
if check.exists():
    info = pd.read_pickle('info.pkl')
    print('Leyendo datos de info.pkl')
else:
    print('Creando info.pkl')
    datos = pd.read_pickle("../exploratorio/dict_dataframes.pkl")
    info = {}
    info['biorreactores'] = {}
    info['centrifugas'] = {}
    info['Fases producción'] ={}
    info['Cinéticos IPC'] = {}
    info['Fases test'] = {}

    biorreactores = info['biorreactores']
    centrifugas = info['centrifugas']
    fases = info['Fases producción']
    cineticos = info['Cinéticos IPC']
    fases_test = info['Fases test']
    
    for k in datos.keys():
        print(k)
        if 'Bio' in k:
            biorreactores[k.split('/')[0].split('.')[0].split(' ')[-1]] = datos[k]
        elif 'Cen' in k:
            centrifugas[k.split('/')[0].split('.')[0].split(' ')[-1]] = datos[k]
        elif 'Fases' in k and 'test' not in k:
            fases[k.split('/')[1]] = datos[k]
        elif 'test' in k:
            fases_test[k.split('/')[1]] = datos[k]
        elif 'Cinéticos' in k:
            cineticos[k.split('/')[1]] = datos[k]
        else:
            info[k.split('/')[0].split('.')[0]] = datos[k]
            
    # Hay que corregir la forma del dataframe de fases de producción de preinóculo
            
    new_columns = ['LOTE', 'Fecha/hora inicio', 'Fecha/hora fin', 'pH línea 1',
        'pH línea 2', 'pH línea 3', 'Turbidez línea 1', 'Turbidez línea 2', 'Turbidez línea 3', 'Línea usada línea 1',
        'Línea usada línea 2', 'Línea usada línea 3']
    info['Fases producción']['Preinóculo'].columns = new_columns
    info['Fases producción']['Preinóculo'] = info['Fases producción']['Preinóculo'].loc[1:]
    
    with open('info.pkl', 'wb') as file:
        pickle.dump(info, file)
        print('Info creado')

Leyendo datos de info.pkl


La función saca bien los datos pero info está mal, tiene NaN donde no debe tenerlos. Hay que volver a ejecutar las lecturas en info.

In [14]:
from utils import limpiar_string_lote

input = []

for _,row in info['OF 123456 v02'].iterrows():
    
    orden = row['Orden']
    lote = limpiar_string_lote(row['Lote'])
    cant_entregada = row['Cantidad entregada']
    
    prod_cultivo = info['Fases producción']['Cultivo final'][info['Fases producción']['Cultivo final']['LOTE'] == lote]
    
    if len(prod_cultivo) == 0: # se trata de un lote de test
        continue
    
    producto_1 = list(prod_cultivo['Producto 1'].values)
    
    # hay que tener en cuenta si el lote tiene padre o no para buscar los datos de preinóculo e inóculo del padre
    if prod_cultivo['Orden en el encadenado'].values == 2:
        lote_parental = prod_cultivo['LOTE parental'].values[0]
        prod_preinoculo = info['Fases producción']['Preinóculo'][info['Fases producción']['Preinóculo']['LOTE'] == lote_parental]
        prod_inoculo = info['Fases producción']['Inóculo'][info['Fases producción']['Inóculo']['LOTE'] == lote_parental]
        
    # FALTA IMPLEMENTAR IF LOTE PARENTAL == 3
    
    else:
        prod_preinoculo = info['Fases producción']['Preinóculo'][info['Fases producción']['Preinóculo']['LOTE'] == lote]
        prod_inoculo = info['Fases producción']['Inóculo'][info['Fases producción']['Inóculo']['LOTE'] == lote]
        
    input_prod = [
    list(prod_preinoculo['pH línea 1'].values), list(prod_preinoculo['pH línea 2'].values), list(prod_preinoculo['pH línea 3'].values), 
    list(prod_preinoculo['Turbidez línea 1'].values), list(prod_preinoculo['Turbidez línea 2'].values), list(prod_preinoculo['Turbidez línea 3'].values), 
    list(prod_preinoculo['Línea usada línea 1'].values), list(prod_preinoculo['Línea usada línea 2'].values), list(prod_preinoculo['Línea usada línea 3'].values), 
    list(prod_inoculo['Volumen de cultivo'].values), list(prod_inoculo['Turbidez inicio cultivo'].values), list(prod_inoculo['Turbidez final culttivo'].values), 
    list(prod_inoculo['Viabilidad final cultivo'].values), list(prod_cultivo['Orden en el encadenado'].values), 
    list(prod_cultivo['Volumen de inóculo utilizado'].values), list(prod_cultivo['Turbidez inicio cultivo'].values), 
    list(prod_cultivo['Turbidez fin cultivo'].values), list(prod_cultivo['Viabilidad final cultivo'].values), 
    list(prod_cultivo['Centrifugación 1 turbidez'].values), list(prod_cultivo['Centrifugación 2 turbidez'].values), 
    list(prod_cultivo['Producto 2'].values)
]
    
    print(input_prod,'-->', producto_1)

[[], [], [], [], [], [], [], [], [], [nan], [nan], [32.800000000000004], [134400000], [1], [82.4], [17.28], [91.2], [184000000], [nan], [nan], [6.0]] --> [1747.92]
[[], [], [], [], [], [], [], [], [], [nan], [nan], [32.800000000000004], [134400000], [1], [80.4], [18.8], [91.2], [181600000], [nan], [nan], [6.56]] --> [1676.16]
[[], [], [], [], [], [], [], [], [], [nan], [nan], [27.84], [115200000], [1], [66.4], [16.16], [86.4], [248000000], [nan], [nan], [8.08]] --> [1928.496]
[[], [], [], [], [], [], [], [], [], [nan], [nan], [27.84], [115200000], [1], [85.60000000000001], [18.48], [83.2], [229600000], [nan], [nan], [5.920000000000001]] --> [1782.8000000000002]
[[5.496], [5.504], [5.5200000000000005], [28.32], [27.92], [32], [1], [1], [0], [155.20000000000002], [18.24], [31.680000000000003], [106400000], [1], [77.60000000000001], [17.12], [74.4], [132800000], [26.56], [20.880000000000003], [2.9600000000000004]] --> [1861.84]
[[5.496], [5.504], [5.5200000000000005], [28.32], [27.92], [3